# Scans Along One Dimension

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

from bluesky import RunEngine
from bluesky.plans import scan, relative_scan, list_scan
from bluesky.callbacks import LiveTable, LivePlot

# Set up simulated hardware.
from bluesky.examples import motor, SynGauss
det = SynGauss('det', motor, 'motor', center=0, Imax=1,
               noise='uniform', sigma=1, noise_multiplier=0.1)

RE = RunEngine({})

### Setting up Subscriptions

In [0]:
table = LiveTable(['det', 'motor_setpoint', 'motor'])
plot = LivePlot('det', 'motor')

In [0]:
s = [table, plot]

## Data Acquisition

### Executing `scan`, `relative_scan`, and `list_scan`

In [0]:
# scan a motor from -5 to 5, taking 10 equally-spaced readings of 'det'
RE(scan([det], motor, -5, 5, 10), s)

In [0]:
# scan a motor from -5 to 5 *relative to its current position*
RE(relative_scan([det], motor, -5, 5, 10), s)

In [0]:
# scan a motor through a list of user-specified positions
RE(list_scan([det], motor, [1, 1, 2, 3, 5, 8]), s)

### Using `print_summary` to inspect a plan without executing it (one approach to "simulation mode")

In [0]:
from bluesky.plan_tools import print_summary

print_summary(scan([det], motor, -1, 1, 5))

In [0]:
print_summary(relative_scan([det], motor, -1, 1, 5))

### Composing plans

In [0]:
def rough_and_fine(detectors, motor, start, stop, rough=10, fine=100):
    "Scan a domain twice, first with widely-spaced and then finely-spaced points."
    yield from scan(detectors, motor, start, stop, num=rough)
    yield from scan(detectors, motor, start, stop, num=fine)

In [0]:
plot = LivePlot('det', 'motor')
RE(rough_and_fine([det], motor, -10, 10), [table, plot])

## Exercises


1. All the examples in this notebooks scan from "left" to "right" (negative to positive). Invoke ``scan`` in a way that scans from right to left.
2. Define a variant of ``rough_and_fine`` that scans relative to the current motor position.
3. Use ``list_scan`` to sample quadratically-spaced points.